In [1]:
from ultralytics import YOLO

def main():
    model = YOLO("yolov8n-seg.pt")   
    model.train(
        data="data.yaml",
        imgsz=640,
        epochs=50,
        batch=16,
        device="cpu",
        workers=4,
        cache=True
    )

if __name__ == "__main__":
    main()


Ultralytics 8.3.241 🚀 Python-3.13.11 torch-2.9.1+cu128 CPU (Intel Core i7-4790K 4.00GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=512, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train11, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pret

In [2]:
from ultralytics import YOLO

model = YOLO("runs/segment/train10/weights/best.pt")
metrics = model.val(data="data.yaml", split="test", imgsz=640)
print(metrics)


Ultralytics 8.3.241 🚀 Python-3.13.11 torch-2.9.1+cu128 CPU (Intel Core i7-4790K 4.00GHz)
YOLOv8n-seg summary (fused): 85 layers, 3,258,259 parameters, 0 gradients, 11.3 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 82.0±38.7 MB/s, size: 35.7 KB)
val: Scanning /home/pyfur1n/Documents/python-nn/Car-Damage detection.v1i.yolov8-obb/dataset/test/labels.cache... 486 images, 0 backgrounds, 21 corrupt: 100% ━━━━━━━━━━━━ 486/486 814.7Kit/s 0.0s
val: /home/pyfur1n/Documents/python-nn/Car-Damage detection.v1i.yolov8-obb/dataset/test/images/akhand_b43_231_jpg.rf.af420168a18dda626057f4d2de807892.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [1.0399169]
val: /home/pyfur1n/Documents/python-nn/Car-Damage detection.v1i.yolov8-obb/dataset/test/images/akhand_b43_232_jpg.rf.8265b063b0568a077ab74af86092c2da.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [1.2847873 1.0455221]
val: /home/pyfur1n/Documents/python-nn/Car-Damage detecti

In [3]:
import cv2
import numpy as np
from ultralytics import YOLO
from pathlib import Path

def severity_by_ratio(ratio: float) -> str:
    # пороги примерные — потом подстроишь под ваш проект
    if ratio < 0.01:
        return "low"
    elif ratio < 0.05:
        return "medium"
    else:
        return "high"

def infer_one(image_path: str, weights: str, conf: float = 0.25, out_dir: str = "pred"):
    out_dir = Path(out_dir)
    out_dir.mkdir(parents=True, exist_ok=True)

    model = YOLO(weights)

    img = cv2.imread(image_path)
    if img is None:
        raise FileNotFoundError(f"Не удалось прочитать изображение: {image_path}")

    H, W = img.shape[:2]
    img_area = H * W

    res = model.predict(img, imgsz=640, conf=conf, verbose=False)[0]

    overlay = img.copy()
    total_area = 0
    obj_count = 0

    if res.masks is not None:
        masks = res.masks.data.cpu().numpy()  # (N, H, W)
        obj_count = masks.shape[0]

        for m in masks:
            mask = (m > 0.5).astype(np.uint8)
            area = int(mask.sum())
            total_area += area

            # оверлей (красный)
            red = np.zeros_like(img)
            red[:, :, 2] = 255
            overlay = np.where(mask[:, :, None] == 1,
                               cv2.addWeighted(overlay, 0.6, red, 0.4, 0),
                               overlay)

    ratio = total_area / max(img_area, 1)
    sev = severity_by_ratio(ratio)

    text = f"objects={obj_count}  area={total_area} ({ratio:.2%})  severity={sev}"
    cv2.putText(overlay, text, (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2, cv2.LINE_AA)

    out_path = out_dir / (Path(image_path).stem + "_pred.jpg")
    cv2.imwrite(str(out_path), overlay)

    return {
        "image": image_path,
        "objects": obj_count,
        "damage_area_px": total_area,
        "damage_ratio": ratio,
        "severity": sev,
        "saved_to": str(out_path)
    }

if __name__ == "__main__":
    r = infer_one("dataset/test/images/1_jpg.rf.13b15550cec5cabf63fe48bcf17aaa05.jpg", "runs/segment/train10/weights/best.pt")
    print(r)


{'image': 'dataset/test/images/1_jpg.rf.13b15550cec5cabf63fe48bcf17aaa05.jpg', 'objects': 2, 'damage_area_px': 30902, 'damage_ratio': 0.0754443359375, 'severity': 'high', 'saved_to': 'pred/1_jpg.rf.13b15550cec5cabf63fe48bcf17aaa05_pred.jpg'}


In [4]:
import csv
from pathlib import Path

def batch_infer(images_dir="dataset/test/images", weights="runs/segment/train10/weights/best.pt"):
    images_dir = Path(images_dir)
    rows = []

    for img_path in images_dir.glob("*.jpg"):
        rows.append(infer_one(str(img_path), weights, out_dir="pred_test"))

    with open("pred_test/report.csv", "w", newline="", encoding="utf-8") as f:
        w = csv.DictWriter(f, fieldnames=rows[0].keys())
        w.writeheader()
        w.writerows(rows)

    print("Готово: pred_test/report.csv и картинки в pred_test/")

batch_infer()


Готово: pred_test/report.csv и картинки в pred_test/


In [5]:
"""
make_report.py

Берёт обученную YOLOv8(-seg) модель:
1) строит графики метрик/лоссов из runs/.../results.csv
2) сохраняет примеры картинок:
   - где damage ЕСТЬ (по label-файлу не пустой)
   - где damage НЕТ (label пустой или отсутствует)
   + поверх рисует предсказания модели (маски/боксы)

Запуск:
  python make_report.py
"""

from pathlib import Path
import random
import pandas as pd
import matplotlib.pyplot as plt

import cv2
import numpy as np
from ultralytics import YOLO


# -----------------------------
# НАСТРОЙКИ (поменяй под себя)
# -----------------------------
RUN_DIR = Path("runs/segment/train10")              # папка обучения
WEIGHTS = RUN_DIR / "weights/best.pt"               # обученная модель
DATA_YAML = Path("data.yaml")                       # твой data.yaml

TEST_IMAGES_DIR = Path("dataset/test/images")       # тестовые картинки
TEST_LABELS_DIR = Path("dataset/test/labels")       # тестовые разметки (txt)

OUT_DIR = Path("report_out")                        # куда сохранять отчёт
N_EXAMPLES_WITH_DAMAGE = 6
N_EXAMPLES_NO_DAMAGE = 6

IMG_SIZE_FOR_PRED = 640
CONF = 0.25
SEED = 42


# -----------------------------
# УТИЛИТЫ
# -----------------------------
def ensure_dir(p: Path):
    p.mkdir(parents=True, exist_ok=True)


def read_results_csv(run_dir: Path) -> pd.DataFrame:
    csv_path = run_dir / "results.csv"
    if not csv_path.exists():
        raise FileNotFoundError(f"Не найден results.csv: {csv_path}")
    df = pd.read_csv(csv_path)
    df.columns = [c.strip() for c in df.columns]
    return df


def plot_curves(df: pd.DataFrame, out_dir: Path):
    """
    Рисуем 2 графика:
      - losses (все колонки с 'loss')
      - metrics (precision/recall/mAP/f1)
    И сохраняем PNG.
    """
    ensure_dir(out_dir)

    # epoch-колонка может называться по-разному, обычно 'epoch'
    xcol = "epoch" if "epoch" in df.columns else df.columns[0]

    # Losses
    loss_cols = [c for c in df.columns if "loss" in c.lower()]
    if loss_cols:
        plt.figure()
        for c in loss_cols:
            plt.plot(df[xcol], df[c], label=c)
        plt.xlabel("epoch")
        plt.ylabel("loss")
        plt.title("Loss curves")
        plt.grid(True)
        plt.legend()
        plt.tight_layout()
        plt.savefig(out_dir / "loss_curves.png", dpi=160)
        plt.close()

    # Metrics
    metric_cols = [c for c in df.columns if any(k in c.lower() for k in ["map", "precision", "recall", "f1"])]
    if metric_cols:
        plt.figure()
        for c in metric_cols:
            plt.plot(df[xcol], df[c], label=c)
        plt.xlabel("epoch")
        plt.ylabel("metric")
        plt.title("Metrics curves")
        plt.grid(True)
        plt.legend()
        plt.tight_layout()
        plt.savefig(out_dir / "metrics_curves.png", dpi=160)
        plt.close()


def label_path_for_image(img_path: Path, labels_dir: Path) -> Path:
    # YOLO labels: same stem, .txt
    return labels_dir / f"{img_path.stem}.txt"


def has_damage_label(img_path: Path, labels_dir: Path) -> bool:
    """
    Damage есть, если label-файл существует и не пустой (есть хотя бы одна строка).
    """
    lp = label_path_for_image(img_path, labels_dir)
    if not lp.exists():
        return False
    txt = lp.read_text(encoding="utf-8", errors="ignore").strip()
    return len(txt) > 0


def pick_examples(images_dir: Path, labels_dir: Path, n_yes: int, n_no: int, seed: int = 42):
    imgs = sorted(list(images_dir.glob("*.jpg"))) + sorted(list(images_dir.glob("*.png"))) + sorted(list(images_dir.glob("*.jpeg")))
    if not imgs:
        raise FileNotFoundError(f"В папке нет изображений: {images_dir}")

    yes = [p for p in imgs if has_damage_label(p, labels_dir)]
    no = [p for p in imgs if not has_damage_label(p, labels_dir)]

    random.seed(seed)
    random.shuffle(yes)
    random.shuffle(no)

    return yes[:n_yes], no[:n_no]


def bgr_to_rgb(img_bgr: np.ndarray) -> np.ndarray:
    return cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)


def save_side_by_side(original_bgr: np.ndarray, pred_bgr: np.ndarray, out_path: Path, title: str = ""):
    """
    Сохраняет картинку "оригинал | предсказание" через matplotlib.
    """
    ensure_dir(out_path.parent)

    orig_rgb = bgr_to_rgb(original_bgr)
    pred_rgb = bgr_to_rgb(pred_bgr)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(orig_rgb)
    plt.axis("off")
    plt.title("Original")

    plt.subplot(1, 2, 2)
    plt.imshow(pred_rgb)
    plt.axis("off")
    plt.title("Prediction")

    if title:
        plt.suptitle(title)

    plt.tight_layout()
    plt.savefig(out_path, dpi=160)
    plt.close()


def run_predict_and_save(model: YOLO, img_path: Path, out_path: Path, conf: float, imgsz: int, title: str):
    img_bgr = cv2.imread(str(img_path))
    if img_bgr is None:
        raise FileNotFoundError(f"Не удалось прочитать: {img_path}")

    # predict
    res = model.predict(img_bgr, conf=conf, imgsz=imgsz, verbose=False)[0]

    # res.plot() возвращает BGR numpy с наложенными масками/боксами
    pred_bgr = res.plot()

    save_side_by_side(img_bgr, pred_bgr, out_path, title=title)


# -----------------------------
# MAIN
# -----------------------------
def main():
    ensure_dir(OUT_DIR)

    # 1) Графики по results.csv
    print("[1/3] Читаю results.csv и строю графики...")
    df = read_results_csv(RUN_DIR)
    plot_curves(df, OUT_DIR)
    print(f"  -> сохранено: {OUT_DIR / 'loss_curves.png'} (если были loss-колонки)")
    print(f"  -> сохранено: {OUT_DIR / 'metrics_curves.png'} (если были metric-колонки)")

    # 2) Выбор примеров YES/NO по разметке
    print("[2/3] Выбираю примеры изображений (damage / no-damage) по labels...")
    yes_imgs, no_imgs = pick_examples(TEST_IMAGES_DIR, TEST_LABELS_DIR, N_EXAMPLES_WITH_DAMAGE, N_EXAMPLES_NO_DAMAGE, seed=SEED)

    print(f"  damage examples: {len(yes_imgs)}")
    print(f"  no-damage examples: {len(no_imgs)}")

    # 3) Прогон модели и сохранение примеров
    print("[3/3] Делаю предсказания и сохраняю примеры...")
    model = YOLO(str(WEIGHTS))

    ex_dir_yes = OUT_DIR / "examples_damage"
    ex_dir_no = OUT_DIR / "examples_no_damage"
    ensure_dir(ex_dir_yes)
    ensure_dir(ex_dir_no)

    for i, p in enumerate(yes_imgs, 1):
        out_path = ex_dir_yes / f"{i:02d}_{p.stem}.png"
        run_predict_and_save(
            model=model,
            img_path=p,
            out_path=out_path,
            conf=CONF,
            imgsz=IMG_SIZE_FOR_PRED,
            title=f"GT: damage=yes | file={p.name}"
        )

    for i, p in enumerate(no_imgs, 1):
        out_path = ex_dir_no / f"{i:02d}_{p.stem}.png"
        run_predict_and_save(
            model=model,
            img_path=p,
            out_path=out_path,
            conf=CONF,
            imgsz=IMG_SIZE_FOR_PRED,
            title=f"GT: damage=no | file={p.name}"
        )

    # (опционально) оценка метрик на test в консоль
    print("\nДополнительно: считаю метрики на test (может занять время на CPU)...")
    metrics = model.val(data=str(DATA_YAML), split="test", imgsz=IMG_SIZE_FOR_PRED)
    print(metrics)

    print("\nГотово! Смотри папку:", OUT_DIR.resolve())


if __name__ == "__main__":
    main()


[1/3] Читаю results.csv и строю графики...
  -> сохранено: report_out/loss_curves.png (если были loss-колонки)
  -> сохранено: report_out/metrics_curves.png (если были metric-колонки)
[2/3] Выбираю примеры изображений (damage / no-damage) по labels...
  damage examples: 6
  no-damage examples: 0
[3/3] Делаю предсказания и сохраняю примеры...

Дополнительно: считаю метрики на test (может занять время на CPU)...
Ultralytics 8.3.241 🚀 Python-3.13.11 torch-2.9.1+cu128 CPU (Intel Core i7-4790K 4.00GHz)
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2066.9±468.4 MB/s, size: 46.4 KB)
val: Scanning /home/pyfur1n/Documents/python-nn/Car-Damage detection.v1i.yolov8-obb/dataset/test/labels.cache... 486 images, 0 backgrounds, 21 corrupt: 100% ━━━━━━━━━━━━ 486/486 993.4Kit/s 0.0s
val: /home/pyfur1n/Documents/python-nn/Car-Damage detection.v1i.yolov8-obb/dataset/test/images/akhand_b43_231_jpg.rf.af420168a18dda626057f4d2de807892.jpg: ignoring corrupt image/label: non-normalized or out of bounds co

In [6]:
!pip install pandas

In [7]:
!pip install streamlit ultralytics opencv-python pillow numpy

In [8]:
# app.py
# Streamlit-приложение: загрузка фото -> "есть damage или нет" + визуализация области + confidence
#
# Запуск:
#   pip install streamlit ultralytics opencv-python pillow numpy
#   streamlit run app.py
#
# ВАЖНО: поменяй MODEL_PATH на путь к твоему best.pt

import streamlit as st
from ultralytics import YOLO
from PIL import Image
import numpy as np
import cv2


# -------------------- НАСТРОЙКИ --------------------
MODEL_PATH = "runs/segment/train10/weights/best.pt"  # <-- поменяй под себя
DEFAULT_IMGSZ = 640
DEFAULT_CONF = 0.25


# -------------------- ЗАГРУЗКА МОДЕЛИ --------------------
@st.cache_resource
def load_model(model_path: str):
    return YOLO(model_path)


def bgr_to_rgb(img_bgr: np.ndarray) -> np.ndarray:
    return cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)


def get_confidences(result):
    """
    Возвращает список confidence для найденных объектов.
    Для YOLOv8: result.boxes.conf -> tensor shape (N,)
    """
    if result.boxes is None or result.boxes.conf is None:
        return []
    return result.boxes.conf.cpu().numpy().tolist()


def main():
    st.set_page_config(page_title="Car Damage Detector", layout="centered")
    st.title("🚗 Car Damage Detector (YOLOv8)")

    st.sidebar.header("Настройки")
    conf_th = st.sidebar.slider("Порог confidence (conf)", 0.0, 1.0, DEFAULT_CONF, 0.01)
    imgsz = st.sidebar.selectbox("Размер изображения (imgsz)", [512, 640, 768, 1024], index=1)
    show_overlay = st.sidebar.checkbox("Показывать область (оверлей)", value=True)

    # Загружаем модель
    try:
        model = load_model(MODEL_PATH)
    except Exception as e:
        st.error(f"Не удалось загрузить модель по пути: {MODEL_PATH}\n\nОшибка: {e}")
        st.stop()

    uploaded = st.file_uploader("Загрузи фото автомобиля (jpg/png)", type=["jpg", "jpeg", "png"])

    if uploaded is None:
        st.info("Загрузи изображение, чтобы получить результат.")
        return

    # PIL -> numpy (RGB)
    img_pil = Image.open(uploaded).convert("RGB")
    img_rgb = np.array(img_pil)

    # Ultralytics принимает numpy (RGB тоже ок), но plot() вернёт BGR
    st.image(img_pil, caption="Исходное изображение", use_container_width=True)

    # Предсказание
    results = model.predict(img_rgb, conf=conf_th, imgsz=imgsz, verbose=False)
    r = results[0]

    confs = get_confidences(r)
    n = len(confs)

    # Логика "есть дамаг или нет"
    if n == 0:
        st.success("✅ Повреждения НЕ обнаружены")
        st.write("Объекты: 0")
    else:
        best_conf = float(max(confs))
        st.warning("⚠️ Обнаружено повреждение (damage)")
        st.write(f"Объекты: **{n}**")
        st.write(f"Максимальная уверенность: **{best_conf:.2f}**")

        # Табличка по объектам
        st.subheader("Уверенность по каждому найденному объекту")
        for i, c in enumerate(confs, 1):
            st.write(f"{i}. confidence = **{float(c):.2f}**")

    # Визуализация области (маски/боксы)
    if show_overlay:
        plotted_bgr = r.plot()  # ndarray BGR
        plotted_rgb = bgr_to_rgb(plotted_bgr)
        st.image(plotted_rgb, caption="Результат (маски/области + confidence)", use_container_width=True)

    # Дополнительно: показать маски есть/нет (для сегментации)
    if r.masks is None:
        st.caption("Маски: нет (res.masks is None)")
    else:
        st.caption(f"Маски: {r.masks.data.shape[0]} шт.")


if __name__ == "__main__":
    main()


2025-12-24 03:11:52.840 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-24 03:11:52.842 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-24 03:11:53.126 
  command:

    streamlit run /home/pyfur1n/Documents/python-nn/Car-Damage detection.v1i.yolov8-obb/.venv/lib/python3.13/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-12-24 03:11:53.126 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-24 03:11:53.127 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-24 03:11:53.127 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-24 03:11:53.128 Thread 'MainThread': m

In [9]:
!streamlit run app.py


  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8502
  Network URL: http://10.101.115.21:8502

  Stopping...
^C
